Importing Libraries 

In [ ]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing.image import ImageDataGenerator #image generator label data based on the dir the image in contained in
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

from keras.utils import get_file, image_dataset_from_directory
import pathlib

: 

Loading images using image_dataset_from_directory and setting batch_size at 32

In [ ]:
dataset  = image_dataset_from_directory(
    'datasets/chest_xray',
    color_mode='rgb',
    batch_size = 128,
    image_size=(256,256)
)

: 

Getting the shape of the data and the labels in the class

In [ ]:
for data, labels in dataset.take(1):
    print(data.shape)
    print(labels.shape)

: 

printing images from the training set 

Normalizing and augmenting dataset

In [ ]:
test_image_generator = ImageDataGenerator(rescale=1./255)
val_image_generator = ImageDataGenerator(rescale=1./255)
train_image_generator = ImageDataGenerator(rescale=1./255)

test_datagen = test_image_generator.flow_from_directory(
    'datasets/chest_xray/',
    target_size=(256,256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    classes=['test'],
    shuffle=False)
    
val_datagen = val_image_generator.flow_from_directory(
    'datasets/chest_xray/val/',
    target_size=(256,256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    shuffle=True)

train_datagen = val_image_generator.flow_from_directory(
    'datasets/chest_xray/train/',
    target_size=(256,256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='binary',
    shuffle=True)

: 

In [ ]:
dataset_datagen = ImageDataGenerator(
    rotation_range= 0.5,
    width_shift_range= 0.2,
    height_shift_range= 0.5,
    horizontal_flip=True,
    vertical_flip=True,
    rescale= 1./255
)

train_datagen = dataset_datagen.flow_from_directory(
    'datasets/chest_xray/train',
    target_size=(256,256),
    color_mode='grayscale',
    batch_size=128,
    class_mode='binary',
    shuffle=True)


: 

Setting up the CNN

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, 
                 kernel_size=3, 
                  
                 activation='relu', 
                 input_shape=(256,256, 1)
                 )
        )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=32, 
                 kernel_size=5, 
                 activation='relu', 
                 # input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
                 )
        )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=64, 
                 kernel_size=7, 
                
                 activation='relu', 
                 #input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
                 )
        )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, 
                 kernel_size=9, 
                
                 activation='relu', 
                 # input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
                 )
        )
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128,
                 activation = 'relu'
))
model.add(Dense(1, 
                activation = 'sigmoid'))
model.summary()

: 

In [ ]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'Adam',
    metrics = ['accuracy']
)

: 

Evaluating the model based on the images from the val folder

In [ ]:
STEP_SIZE_TRAIN=train_datagen.n//train_datagen.batch_size
STEP_SIZE_VALID=val_datagen.n//val_datagen.batch_size

: 

In [ ]:
history = model.fit(
    train_datagen,
    epochs = 15,
    steps_per_epoch=STEP_SIZE_TRAIN,
    validation_data=val_datagen,
    validation_steps=STEP_SIZE_VALID,
)

: 

In [ ]:
history = model.evaluate(
    val_datagen,
    batch_size = 128,
)

: 

In [ ]:
print('The testing accuracy is :',history[1]*100, '%')

: 

In [ ]:
def plotImages(images_arr, probabilities = False):
    fig, axes = plt.subplots(len(images_arr), 1, figsize=(5,len(images_arr) * 3))
    if probabilities is False:
      for img, ax in zip( images_arr, axes):
          ax.imshow(img)
          ax.axis('off')
    else:
      for img, probability, ax in zip( images_arr, probabilities, axes):
          ax.imshow(img)
          ax.axis('off')
          if probability > 0.5:
              ax.set_title("%.2f" % (probability*100) + "% normal")
          else:
              ax.set_title("%.2f" % ((1-probability)*100) + "% pnuemonia")
    plt.show()

: 

In [ ]:
probabilities = model.predict(test_datagen)
probabilities = probabilities.reshape(-1)
probabilities = probabilities.tolist()

sample_testing_images, _ = next(test_datagen)
plotImages(sample_testing_images, probabilities = probabilities)

: 